CA02: This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm.

In this assignment you will ...
1. Complete the code such a way that it works correctly with this given parts of the program.
2. Explain as clearly as possible what each part of the code is doing. Use "Markdown" texts and code commenting to explain the code

IMPORTANT NOTE:

The path of your data folders 'train-mails' and 'test-mails' must be './train-mails' and './test-mails'. This means you must have your .ipynb file and these folders in the SAME FOLDER in your laptop or Google Drive. The reason for doing this is, this way the peer reviewes and I would be able to run your code from our computers using this exact same relative path, irrespective of our folder hierarchy.

In [10]:
import os
import numpy as np
from collections import Counter

# Import all other necessary libraries. Your code below ...
from google.colab import drive
drive.mount("/content/drive")
#

Mounted at /content/drive


In [2]:
def make_Dictionary(root_dir):
  all_words = []#empty dic for all our words found in the emails
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)] #looks in root dir to find all the email files
  for mail in emails:
    with open(mail) as m:
      for line in m:
        words = line.split()#for each email oppen and read it by line
        all_words += words #collecting all words and adding it to all words
  dictionary = Counter(all_words)#counts how many times each word shows up
  list_to_remove = list(dictionary)#creates a list of words in the dictionary

  for item in list_to_remove: #loop checks each word in the list to remove
    if item.isalpha() == False:#if it has non letter characters like numbers or symbols delete it
      del dictionary[item]
    elif len(item) == 1:#if the word is one letter long also delete it
      del dictionary[item]
  dictionary = dictionary.most_common(3000)#most common is a method in counter that retrieves the most popopular 3000 words
  return dictionary



In [3]:
def extract_features(mail_dir):#mail dir will be a folder full of emails
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]#look inside these folders and make a list of paths to evbery email file
  features_matrix = np.zeros((len(files),3000))#rows are number of emails and columns are gonna be the 3000 popular words we care about
  train_labels = np.zeros(len(files))#another list of zeroes, one number per email, 0 not spam 1 spam
  count = 1;#unused helper according to AI, not needed
  docID = 0;#which email am i working on right now?
  for fil in files:#or each email file
    with open(fil) as fi:#open the email so we can read it
      for i, line in enumerate(fi):#i is the line number, line is the actual text, so read the email line by line
        if i ==2:#so start looking at line 3 skip the first two, this is cuz counting starts at 0 and we are going straight to the body of the email
          words = line.split()#split the sentence into seperate words
          for word in words:#look at eaech word
            wordID = 0#where which column the word belongs to
            for i, d in enumerate(dictionary): #d is the actual word here, go through the dictionary of most popular 3k words
              if d[0] == word: #if the word matches one of our 3000
                wordID = i #remember which column it belongs to
                features_matrix[docID,wordID] = words.count(word) #count how many times it shows up in this email
      train_labels[docID] = 0; #assume it is not spam
      filepathTokens = fil.split('/') #break the file path into pieces, i dont really get why
      lastToken = filepathTokens[len(filepathTokens)-1] #get the file name
      if lastToken.startswith("spmsg"): #if the file name starts with spmsg
        train_labels[docID] = 1;#mark as spam
        count = count + 1#move onto the next and count it
      docID = docID + 1
  return features_matrix, train_labels     #return the big table of word couns, and the list of spam or not spam labels
  #email number will be the rows and the count of words will be the columns

In [15]:
# Enter the "path" of your "train_mails" and "test-mails" FOLDERS in this cell ...
# for example: TRAIN_DIR = '../../train-mails'
#              TEST_DIR = '../../test-mails'
TRAIN_DIR = "/content/drive/MyDrive/train-mails"
TEST_DIR = "/content/drive/MyDrive/test-mails"

In [16]:
dictionary = make_Dictionary(TRAIN_DIR)

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR)#big table of word counts
test_features_matrix, test_labels = extract_features(TEST_DIR)#labels spam or not spam

reading and processing emails from TRAIN and TEST folders


In [ ]:
# In this section enter your code to TRAIN the model using Naive Bayes algorithm, then PREDICT and then evaluate PERFORMANCE (Accuracy)
# Your code below ...
#I will keep this as text to explain what will be done next, as per the output we will be using the Gaussian Bayes  algorithm
#What it does is look at the compilation of words and asks the simple question on whether the email is likely spam or not
#It learns from our 3000 dictionary how many times these words show up in spam emails, and in contrary how many they show up in normal ones
# Your output should look like below if your code is right

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9653846153846154


In [17]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

print("Training Model using Gaussian Naive Bayes algorithm .....")

# 1) create the model
model = GaussianNB()

# 2) train (learn patterns from the training data)
model.fit(features_matrix, labels)

print("Training completed")
print("testing trained model to predict Test Data labels")

# 3) predict labels for the test data
predicted_labels = model.predict(test_features_matrix)

print("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")

# 4) evaluate performance (accuracy)
acc = accuracy_score(test_labels, predicted_labels)
print(acc)

print("================ END OF PROGRAM =================")


Training Model using Gaussian Naive Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9653846153846154
================ END OF PROGRAM =================


======================= END OF PROGRAM =========================

The start of my revised code. AI assisted to find better optimization and other solutions I was not yet privy to.

In [ ]:
import os
import numpy as np
from collections import Counter

# Import all other necessary libraries. Your code below ...
from google.colab import drive
drive.mount("/content/drive")


In [ ]:
from collections import Counter
import os

def make_Dictionary(root_dir):
    all_words = []
    emails = [os.path.join(root_dir, f) for f in os.listdir(root_dir)]

    for mail in emails:
        #this is the first major change, this was done to prevent crashes when emails have weird characters to ignore them
        with open(mail, "r", errors="ignore") as m:
            for line in m:
                words = line.split()
                all_words += words

    dictionary = Counter(all_words)

    #major change 2, in the past code we had the list_to_remove and then the for loop under it
    #the old code deleted the words as we went through them which is seen as unsafe in coding
    #the way around that was list_to_remove= list(dictionary), which was like setting them aside and then deleting them
    #we are maintaining the low count words that pose no risk and making them the safe word list, no need for deletion and risk of loss of data
    dictionary = Counter({
        word: count
        for word, count in dictionary.items()
        if word.isalpha() and len(word) > 1
    })

    dictionary = dictionary.most_common(3000)
    return dictionary


In [18]:
#this optimization was done in assistance with AI to find faster solutions and help reduce the redundant loops in the code
def extract_features(mail_dir, dictionary, n_words=3000):
    """
    Turns each email file into a row of word-count features.
    - mail_dir: folder containing email files
    - dictionary: list like [(word, count), ...] from make_Dictionary
    - n_words: number of dictionary words (default 3000)
    """
    files = [os.path.join(mail_dir, f) for f in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files), n_words), dtype=float)
    labels = np.zeros(len(files), dtype=int)

    #this helps reduce the next few lines of code take minutes (from the original)
    #it does so by seeing a word and finding its column
    word_to_idx = {w: i for i, (w, _) in enumerate(dictionary[:n_words])}

    for docID, filepath in enumerate(files):
        # label from filename
        filename = os.path.basename(filepath)
        labels[docID] = 1 if filename.startswith("spmsg") else 0

        # read whole email text (more robust than i==2)
        with open(filepath, "r", errors="ignore") as f:
            text = f.read()

        words = text.split()
        counts = Counter(words)

        # fill row using counts
        for w, c in counts.items():
            idx = word_to_idx.get(w) #this is what I referred to earlier in finding the columns
            if idx is not None:
                features_matrix[docID, idx] = c

    return features_matrix, labels
#this is faster than the original because we would check for every word, go through the entire dictionary to find where it belongs and then
#go through the email again to count it, inneficient and pointless

In [ ]:
TRAIN_DIR = "/content/drive/MyDrive/train-mails"
TEST_DIR = "/content/drive/MyDrive/test-mails"

In [19]:
dictionary = make_Dictionary(TRAIN_DIR)

features_matrix, labels = extract_features(TRAIN_DIR, dictionary)
test_features_matrix, test_labels = extract_features(TEST_DIR, dictionary)


In [20]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

print("Training Model using Gaussian Naive Bayes algorithm .....")

# 1) create the model
model = GaussianNB()

# 2) train (learn patterns from the training data)
model.fit(features_matrix, labels)

print("Training completed")
print("testing trained model to predict Test Data labels")

# 3) predict labels for the test data
predicted_labels = model.predict(test_features_matrix)

print("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")

# 4) evaluate performance (accuracy)
acc = accuracy_score(test_labels, predicted_labels)
print(acc)

print("================ END OF PROGRAM =================")

Training Model using Gaussian Naive Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9653846153846154
================ END OF PROGRAM =================
